In [12]:
import os
import pandas as pd

In [13]:
import sys
sys.path.append('../Scripts')

In [14]:
from functions import *

In [5]:
prefixList = ["ER064912", "ERR044595"]

In [6]:
OutDir = "/home/junyuchen/1-Projects/WGS-Pipeline/result/testout"

In [7]:
ProkkaDir = os.path.join(OutDir, "prokka")

In [8]:
fnaList, faaList, ffnList = parseProkka(prefixList, ProkkaDir)

In [9]:
Kraken2Dir = os.path.join(OutDir, "kraken2")
if os.path.exists(Kraken2Dir) == 0:
    os.makedirs(Kraken2Dir, 0o777, True)

In [11]:
fnaList

['/home/junyuchen/1-Projects/WGS-Pipeline/result/testout/prokka/ER064912/ER064912.fna',
 '/home/junyuchen/1-Projects/WGS-Pipeline/result/testout/prokka/ERR044595/ERR044595.fna']

In [12]:
!grep ">" /home/junyuchen/1-Projects/WGS-Pipeline/result/testout/prokka/ER064912/ER064912.fna | wc -l

112


In [13]:
database = '/home/junyuchen/3-Resources/Databases/k2_standard_20201202'

In [21]:
## Run kraken2
def RunKraken2(fasta, database, prefix, OutDir, threads):
    cmd = "kraken2 -db " + database + " " + fasta + " --report " + os.path.join(OutDir, prefix + "_kraken2.tsv") + " --threads " + str(threads)
    subprocess.call(cmd, shell=True)
#Run kraken2 in parallel
def RunKraken2Parallel(fileList, database, prefixList, OutDir, threads, jobs):
    pool = Pool(processes=jobs)
    pool.starmap(RunKraken2, zip(fileList, repeat(database), prefixList, repeat(OutDir), repeat(threads)))
    pool.close()
    pool.join()
    pool.terminate()

In [22]:
RunKraken2Parallel(fnaList, database, prefixList, OutDir, 2, 2)

In [15]:
from Bio import Entrez
Entrez.email = "jy.chen1@siat.ac.cn"

In [16]:
import pandas as pd

In [17]:
#parse Kraken2 result
def parseKraken2(report):
    df = pd.read_table(report)
    df.columns = ["%", "reads", "lreads", "lvl", "tid", "name"]
    df1 = df.loc[df["lvl"] == "S"]
    df2 = df1.loc[df1["%"] == df1["%"].max()]
    df2 = df2.reset_index()
    tid = df2["tid"][0]
    handle = Entrez.efetch(db="Taxonomy", id=str(tid) , retmode="xml")
    records = Entrez.read(handle)
    taxa = records[0]["Lineage"] + "; " + records[0]["ScientificName"]
    return tid, taxa

In [33]:
tidList = []
taxaList = []
for prefix in prefixList:
    filePath = os.path.join(OutDir, prefix + "_kraken2.tsv")
    print(filePath)
    (tid, taxa) = parseKraken2(filePath)
    tidList.append(tid)
    taxaList.append(taxa)

/home/junyuchen/1-Projects/WGS-Pipeline/result/testout/ER064912_kraken2.tsv
/home/junyuchen/1-Projects/WGS-Pipeline/result/testout/ERR044595_kraken2.tsv


In [34]:
tidList

[1280, 1280]

In [35]:
taxaList

['cellular organisms; Bacteria; Terrabacteria group; Firmicutes; Bacilli; Bacillales; Staphylococcaceae; Staphylococcus; Staphylococcus aureus',
 'cellular organisms; Bacteria; Terrabacteria group; Firmicutes; Bacilli; Bacillales; Staphylococcaceae; Staphylococcus; Staphylococcus aureus']

In [1]:
InDir = "/home/junyuchen/1-Projects/WGS-Pipeline/result/210202_WGS/kraken2"

In [18]:
df = pd.DataFrame()
for subdir, dirs, files in os.walk(InDir):
    for file in files:
        if file.endswith("_kraken2.tsv"):
            filePath = os.path.join(subdir, file)
            prefix = file.replace("_kraken2.tsv", "")
            tid, taxa = parseKraken2(filePath)
            df = df.append({"SampleID":prefix, "tid":tid, "taxa":taxa}, ignore_index=True)

URLError: <urlopen error [Errno -2] Name or service not known>

In [ ]:
df